In [2]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

from geeml.utils import eeprint

In [3]:
import os
os.chdir(r"C:\Users\coach\myfiles\postdoc\Fire\code\Burn_Area_Mapping\src\components")
os.getcwd()

'C:\\Users\\coach\\myfiles\\postdoc\\Fire\\code\\Burn_Area_Mapping\\src\\components'

In [4]:
from src.logger import logging
from data_preprocessing import preProcessyCollection
from data_preprocessing import preProcessXCollection
from data_extraction import extractXy, extractDataset
%load_ext autoreload

In [5]:
from exception import customException
import sys

In [6]:
%autoreload 2

In [7]:
import geemap
Map = geemap.Map()
fc = ee.FeatureCollection("projects/ee-geethensingh/assets/postdoc/proj_fired_south_africa_to2021182_events")
Map.addLayer(ee.Feature(fc.first()), {'color': 'red'}, "Fire Test")
Map.centerObject(fc.first(), 12)
Map

Map(center=[-31.374999751707104, 29.510986261502627], controls=(WidgetControl(options=['position', 'transparen…

In [8]:
fireEvents = fc.map(lambda ft: ft.set('system:time_start', ee.Date(ft.get('ig_date')))
                    .set('system:time_end', ee.Date(ft.get('last_date')))).filterDate('2019-01-01', '2020-01-01')

In [13]:
supportedSensors = {'Sentinel-2': ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED"),
                    'LANDSAT_4': ee.ImageCollection("LANDSAT/LT04/C02/T1_L2"),
                    'LANDSAT_5': ee.ImageCollection("LANDSAT/LT05/C02/T1_L2"),
                    'LANDSAT_7': ee.ImageCollection("LANDSAT/LE07/C02/T1_L2"),
                    'LANDSAT_8': ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"),
                    'LANDSAT_9': ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")}

# Filter countries by geometry
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
geometry = ee.Geometry.Point([24.06353794842853, -29.732969740562062])
sa = countries.filterBounds(geometry)
sa_geo = sa.geometry()

image = supportedSensors.get('LANDSAT_8').filterBounds(sa_geo).filterDate('2019-01-01', '2020-01-01').first()
eeprint(image)

# Get satellite name
sensor = image.get('SPACECRAFT_ID').getInfo()
if sensor is None:
    sensor = 'Sentinel-2'

eeprint(sensor)

endDate = image.date()
startDate = endDate.advance(ee.Number(8*-7),'day')
# Define datasets, get xweeks nImages prior to image (i.e., get time series info)
xImages =  preProcessXCollection(collection = supportedSensors.get(sensor),
                                    region = image.geometry(), startDate = ee.Date('2019-01-01'), endDate = ee.Date('2020-01-01'))

In [14]:
xImages.size()

In [21]:
extractDataset('LANDSAT_8', 'South Africa', ee.Date('2019-01-01'), ee.Date('2020-01-01'), fireEvents, 16, 'extract_test.csv')

  0%|          | 0/1742 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
endDate = ee.Date(fc.first().get('ig_date'))
startDate = endDate.advance(8*-7,'days')

eeprint(endDate)

In [18]:
t1 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(fireEvent).filterDate(startDate, endDate)
Map.addLayer(t1.sort('CLOUD_COVER', False).first(), {'min': 0, 'max': 0.3}, "L8 VNIR")

from data_preprocessing import cloudMask
cloudMasked = cloudMask('LANDSAT_8', ee.Image(t1.sort('CLOUD_COVER', False).first()))
Map.addLayer(cloudMasked, {'min': 0, 'max': 0.3}, "L8 SWIR")
# eeprint(t1.sort('CLOUD_COVER', False).first())

In [44]:
fireEvent = fc.first().geometry()
X = preProcessXCollection(collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'), region = fireEvent, startDate = startDate, endDate = endDate)
# eeprint(X.first())
eeprint(X.aggregate_array("system:index"))
reducer = ee.Reducer.percentile([5, 25, 50, 75, 95])
L8percentiles = X.select('nbr').reduce(reducer)
# S2percentiles = S2filtered.reduce(reducer)
# Compute temporal variance
L8variance = L8percentiles.reduce(ee.Reducer.variance()).rename('temporal_variance')
# S2variance = S2percentiles.reduce(ee.Reducer.variance())

# Compute image spatial stats
L8stats = L8variance.reduceRegion(reducer = reducer,
                                        geometry = X.geometry(),
                                        scale = 250)
# eeprint(L8stats)
# import pandas as pd
outdf = pd.DataFrame().append(pd.DataFrame([L8stats.getInfo()]))
outdf['Scene_IDs'] = [X.aggregate_array("LANDSAT_PRODUCT_ID").getInfo()]
outdf
#     # features_df = ee.data.computeFeatures({'expression': L8stats, 'fileFormat': 'PANDAS_DATAFRAME'})
#     # features_df.crs = 'EPSG:4326'
#     # outdf = pd.concat([outdf, features_df])
#     row = pd.Series(L8stats.getInfo())
#     print(row)
#     print(row)
#     outdf = pd.concat([outdf, row])

,temporal_variance_p25,temporal_variance_p5,temporal_variance_p50,temporal_variance_p75,temporal_variance_p95,Scene_IDs
0,0.000192,0.000192,0.000192,0.000192,0.003374,"[LC08_L2SP_168082_20210529_20210608_02_T1, LC0..."


In [46]:
# fc = ee.FeatureCollection("projects/ee-geethensingh/assets/postdoc/proj_fired_south_africa_to2021182_events")
extractXy(yCollection= fc.limit(3), Xweeks= 4, filename= 'fired_test.csv')

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:

# Define the date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Convert start and end dates to ee.Date objects
start_date_ee = ee.Date(start_date)
end_date_ee = ee.Date(end_date)

# Define the date to check
date_to_check = '2024-06-15'
date_to_check_ee = ee.Date(date_to_check)

dateRange = ee.DateRange(date_to_check_ee, date_to_check_ee)

print(dateRange.contains(date_to_check_ee).getInfo())

False


In [16]:
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
# Filter countries by geometry
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
geometry = ee.Geometry.Point([24.06353794842853, -29.732969740562062])
sa = countries.filterBounds(geometry)
sa_geo = sa.geometry()

# Define start and end dates
startDate = ee.Date.fromYMD(2018, 7, 4)
endDate = ee.Date.fromYMD(2019, 8, 15)

In [7]:
X = preProcessXCollection(collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'), region = sa_geo, startDate = startDate, endDate = endDate)
eeprint(X.limit(2))
eeprint(X.first().bandNames())